## Day 2

### Part 1 

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

- 1-3 a: abcde
- 1-3 b: cdefg
- 2-9 c: ccccccccc

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

In [1]:
# Import pandas
import pandas as pd

In [2]:
# Read in data
df = pd.read_csv("input_data/Day2.txt", sep=' ', header=None)

In [3]:
df.shape

(1000, 3)

In [4]:
df.head()

,0,1,2
0,2-8,t:,pncmjxlvckfbtrjh
1,8-9,l:,lzllllldsl
2,3-11,c:,ccchcccccclxnkcmc
3,3-10,h:,xcvxkdqshh
4,4-5,s:,gssss


In [5]:
df.columns=['range','letter','password']

In [6]:
df.head()

,range,letter,password
0,2-8,t:,pncmjxlvckfbtrjh
1,8-9,l:,lzllllldsl
2,3-11,c:,ccchcccccclxnkcmc
3,3-10,h:,xcvxkdqshh
4,4-5,s:,gssss


In [7]:
df[['min_range','max_range']]=df['range'].str.split('-', expand=True)
df['letter']=df['letter'].str.split(':', expand=True)

In [8]:
df = df.drop(columns='range')

In [9]:
df['min_range'] = df['min_range'].astype(int)
df['max_range'] = df['max_range'].astype(int)

In [10]:
df.head()

,letter,password,min_range,max_range
0,t,pncmjxlvckfbtrjh,2,8
1,l,lzllllldsl,8,9
2,c,ccchcccccclxnkcmc,3,11
3,h,xcvxkdqshh,3,10
4,s,gssss,4,5


In [11]:
# We'll create a new column which contains the letter count for each password
df['letter_count'] = df.apply(lambda x: x['password'].count(x['letter']), axis=1)

In [12]:
df.head()

,letter,password,min_range,max_range,letter_count
0,t,pncmjxlvckfbtrjh,2,8,1
1,l,lzllllldsl,8,9,7
2,c,ccchcccccclxnkcmc,3,11,11
3,h,xcvxkdqshh,3,10,2
4,s,gssss,4,5,4


In [13]:
# Now create another column which indicates whether password is valid or not
df['valid'] = df.apply(lambda x: x['min_range']<=x['letter_count']<=x['max_range'], axis=1)

In [14]:
df.head()

,letter,password,min_range,max_range,letter_count,valid
0,t,pncmjxlvckfbtrjh,2,8,1,False
1,l,lzllllldsl,8,9,7,False
2,c,ccchcccccclxnkcmc,3,11,11,True
3,h,xcvxkdqshh,3,10,2,False
4,s,gssss,4,5,4,True


In [15]:
# Sum all the valid passwors
df['valid'].sum()

666

### Part 2 

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

- 1-3 a: abcde is valid: position 1 contains a and position 3 does not.
- 1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
- 2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.

How many passwords are valid according to the new interpretation of the policies?

In [16]:
df.head()

,letter,password,min_range,max_range,letter_count,valid
0,t,pncmjxlvckfbtrjh,2,8,1,False
1,l,lzllllldsl,8,9,7,False
2,c,ccchcccccclxnkcmc,3,11,11,True
3,h,xcvxkdqshh,3,10,2,False
4,s,gssss,4,5,4,True


In [17]:
# Let's rework our df for this new problem
df.rename(columns={'min_range':'first_pos', 'max_range':'second_pos'}, inplace=True)

In [18]:
df.drop(columns='letter_count', inplace=True)
df.valid = False

In [19]:
df.head()

,letter,password,first_pos,second_pos,valid
0,t,pncmjxlvckfbtrjh,2,8,False
1,l,lzllllldsl,8,9,False
2,c,ccchcccccclxnkcmc,3,11,False
3,h,xcvxkdqshh,3,10,False
4,s,gssss,4,5,False


In [20]:
# create two new boolean columns indicating whether the first and second position match
df['fp_match'] = df.apply(lambda x: x['password'][x['first_pos']-1]==x['letter'], axis=1)
df['sp_match'] = df.apply(lambda x: x['password'][x['second_pos']-1]==x['letter'], axis=1)

In [21]:
df.head()

,letter,password,first_pos,second_pos,valid,fp_match,sp_match
0,t,pncmjxlvckfbtrjh,2,8,False,False,False
1,l,lzllllldsl,8,9,False,False,False
2,c,ccchcccccclxnkcmc,3,11,False,True,False
3,h,xcvxkdqshh,3,10,False,False,True
4,s,gssss,4,5,False,True,True


In [22]:
df['valid'] = (df['fp_match'] != df['sp_match'])

In [23]:
df.head()

,letter,password,first_pos,second_pos,valid,fp_match,sp_match
0,t,pncmjxlvckfbtrjh,2,8,False,False,False
1,l,lzllllldsl,8,9,False,False,False
2,c,ccchcccccclxnkcmc,3,11,True,True,False
3,h,xcvxkdqshh,3,10,True,False,True
4,s,gssss,4,5,False,True,True


In [24]:
df['valid'].sum()

670